In [145]:
%matplotlib inline
import numpy as np
import pandas as pd
import csv
import geoplotlib
from collections import defaultdict

In [146]:
def readfile(filename):
    data = pd.read_csv(filename)
    return data

# change this to your local file
filename = 'NYNoise.csv'
# read file
data = readfile(filename)


In [147]:
#add a column with the year from the Date
data['year'] = pd.DatetimeIndex(data['Created Date']).year

In [148]:
# filter the dataset. Just keep the rows that belong to year 2016.
data2016 = data[data['year'] == 2016]

In [149]:
data2016.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location,year
170,32316013,01/02/2016 02:52:00 PM,05/23/2016 01:30:00 PM,DEP,Department of Environmental Protection,Noise,Noise: air condition/ventilation equipment (NV1),NaN,11212.0,76 REMSEN AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.661935,-73.930165,"(40.66193535945621, -73.93016485789653)",2016
171,32319864,01/02/2016 08:47:00 AM,01/09/2016 08:30:00 AM,DEP,Department of Environmental Protection,Noise,Noise: Construction Before/After Hours (NM1),NaN,11355.0,43-16 UNION STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,40.754347,-73.821371,"(40.75434676643549, -73.82137094935233)",2016
172,32320142,01/02/2016 09:05:00 PM,01/12/2016 02:45:00 PM,DEP,Department of Environmental Protection,Noise,Noise: air condition/ventilation equipment (NV1),NaN,10025.0,2686 BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,40.798817,-73.968740,"(40.79881676206265, -73.96873982214959)",2016
173,32325036,01/03/2016 07:05:00 AM,01/04/2016 10:40:00 AM,DEP,Department of Environmental Protection,Noise,"Noise, Barking Dog (NR5)",NaN,10312.0,70 RAMONA AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,40.541135,-74.188306,"(40.541135407737606, -74.18830591469624)",2016
174,32328843,01/04/2016 12:23:00 PM,01/06/2016 12:28:00 PM,DEP,Department of Environmental Protection,Noise,Noise: air condition/ventilation equipment (NV1),NaN,10025.0,2528 BROADWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,40.793660,-73.972498,"(40.793660374788146, -73.97249823045138)",2016


In [150]:
#data2016.to_csv("2016.csv")
#data2016=pd.read_csv("2016.csv")

In [151]:
print data2016['Location'].values[0]

(40.66193535945621, -73.93016485789653)


In [152]:
import math
i=0
noise_loc_cluster=[]

latitudes = [float(item) for item in data2016['Latitude'] if not math.isnan(item)]
longitudes = [float(item) for item in data2016['Longitude'] if not math.isnan(item)]

for i in range(len(latitudes)):
    noise_loc_cluster.append([latitudes[i],longitudes[i]])



In [153]:
noise_cluster_list=list(noise_loc_cluster)

In [154]:
from sklearn.cluster import KMeans
clusters = []
for k in range(2, 7):
    cls = KMeans(n_clusters=k)
    cls.fit(noise_cluster_list)
    clusters.append(cls)

In [155]:
print clusters

[KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0), KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0), KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=4, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0), KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0), KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)]


In [156]:
# Export: list of [point_x, point_y, label_2means,label_3means]
export_data = []
for i in range(len(noise_cluster_list)):
    export_data.append([noise_cluster_list[i][0], noise_cluster_list[i][1], clusters[0].labels_[i], clusters[1].labels_[i], clusters[2].labels_[i], clusters[3].labels_[i], clusters[4].labels_[i]])
    
export_data_centers = [cls.cluster_centers_.tolist() for cls in clusters]
export_json = {'centers': export_data_centers, 'data': export_data}

print "centers:", export_data_centers[0]
print "data:", export_data[0]

centers: [[40.70593440223117, -73.98397827167807], [40.776246682974374, -73.88976182253366]]
data: [40.66193535945621, -73.93016485789653, 0, 2, 0, 1, 1]


In [157]:
import json

with open('kmeans.json', 'w') as outfile:
    json.dump(export_json, outfile)